# small convnet from scratch

In [17]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.callbacks import ModelCheckpoint
from keras import backend as K

In [18]:
# dimensions of our images.
img_width, img_height = 299, 299

In [19]:
train_data_dir = 'data/train'
validation_data_dir = 'data/valid'
nb_train_samples = 1037
nb_validation_samples = 444
nb_classes = 3  # number of classes
epochs = 50
batch_size = 16

In [20]:
if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

In [21]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(3))
model.add(Activation('sigmoid'))

In [22]:
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
metrics=['accuracy'])

In [23]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 297, 297, 32)      896       
_________________________________________________________________
activation_6 (Activation)    (None, 297, 297, 32)      0         
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 148, 148, 32)      0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 146, 146, 32)      9248      
_________________________________________________________________
activation_7 (Activation)    (None, 146, 146, 32)      0         
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 73, 73, 32)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 71, 71, 64)        18496     
__________

In [24]:
# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
horizontal_flip=True)

In [25]:
# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1. / 255)

In [26]:
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
class_mode='categorical')

Found 1037 images belonging to 3 classes.


In [27]:
validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
class_mode='categorical')

Found 444 images belonging to 3 classes.


In [28]:
# See: https://github.com/fchollet/keras/issues/5475
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [29]:
# checkpoint
filepath="weights-improvement-{epoch:02d}-{val_acc:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

In [30]:
print("\nStarting to Tune Model\n")
model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
validation_steps=nb_validation_samples // batch_size,
callbacks=callbacks_list)
print('Finished tuning')


Starting to Tune Model

Epoch 1/50
64/64 [==============================] - 221s - loss: 1.0505 - acc: 0.4824 - val_loss: 1.0244 - val_acc: 0.5347
Epoch 2/50
64/64 [==============================] - 195s - loss: 0.9925 - acc: 0.5216 - val_loss: 0.9492 - val_acc: 0.5347
Epoch 3/50
64/64 [==============================] - 195s - loss: 0.9868 - acc: 0.5136 - val_loss: 0.9391 - val_acc: 0.5394
Epoch 4/50
64/64 [==============================] - 195s - loss: 0.9435 - acc: 0.5206 - val_loss: 0.9236 - val_acc: 0.5347
Epoch 5/50
64/64 [==============================] - 194s - loss: 0.9558 - acc: 0.5099 - val_loss: 0.9626 - val_acc: 0.5370
Epoch 6/50
64/64 [==============================] - 195s - loss: 0.9265 - acc: 0.5539 - val_loss: 0.9209 - val_acc: 0.5532
Epoch 7/50
64/64 [==============================] - 194s - loss: 0.9413 - acc: 0.5324 - val_loss: 0.9261 - val_acc: 0.5718
Epoch 8/50
64/64 [==============================] - 194s - loss: 0.9304 - acc: 0.5467 - val_loss: 0.9267 - val_acc

64/64 [==============================] - 196s - loss: 0.8468 - acc: 0.5890 - val_loss: 1.0199 - val_acc: 0.5810
Epoch 34/50
64/64 [==============================] - 194s - loss: 0.8560 - acc: 0.6073 - val_loss: 0.9238 - val_acc: 0.5741
Epoch 35/50
64/64 [==============================] - 195s - loss: 0.8628 - acc: 0.5902 - val_loss: 1.0707 - val_acc: 0.5694
Epoch 36/50
64/64 [==============================] - 196s - loss: 0.8654 - acc: 0.6017 - val_loss: 1.0204 - val_acc: 0.5648
Epoch 37/50
64/64 [==============================] - 197s - loss: 0.8484 - acc: 0.6139 - val_loss: 0.9516 - val_acc: 0.5648
Epoch 38/50
64/64 [==============================] - 196s - loss: 0.8882 - acc: 0.5916 - val_loss: 0.9540 - val_acc: 0.5671
Epoch 39/50
64/64 [==============================] - 194s - loss: 0.8429 - acc: 0.5963 - val_loss: 0.9341 - val_acc: 0.5903
Epoch 40/50
64/64 [==============================] - 196s - loss: 0.8671 - acc: 0.6024 - val_loss: 0.9304 - val_acc: 0.5810
Epoch 41/50
64/64 [=

In [31]:
model.save_weights('second_try.h5')

In [32]:
# serialize model to JSON
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)